In [47]:
import requests 
import time
import json
from bs4 import  BeautifulSoup
import pandas as pd

In [2]:
def get_proxy():
    return requests.get("http://127.0.0.1:5010/get/").json()

In [3]:
def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))

In [4]:
def getHtml(url):
    retry_count = 5
    proxy = get_proxy().get("proxy")
    while retry_count > 0:
        try:
            # 使用代理访问
            html = requests.get(url, proxies={"http": "http://{}".format(proxy)})
            return html
        except Exception:
            retry_count -= 1
            # 删除代理池中代理
            delete_proxy(proxy)
    print('无可用代理')
    return None

In [7]:
with open('place_info.txt', 'r') as f:   
    lines = f.readlines() 
lines_str = str(lines).replace('\'', '"')[2:-2]
city_url_info = json.loads(lines_str)

In [8]:
city_url_info['山东省']

{'link': '/lvyou-shandong.html',
 'place': '674',
 'children': {'济南市': {'link': '/lvyou-jinan.html', 'place_num': '47'},
  '青岛市': {'link': '/lvyou-qingdao.html', 'place_num': '98'},
  '淄博市': {'link': '/lvyou-zibo.html', 'place_num': '48'},
  '枣庄市': {'link': '/lvyou-zaozhuang.html', 'place_num': '32'},
  '东营市': {'link': '/lvyou-dongying.html', 'place_num': '20'},
  '烟台市': {'link': '/lvyou-yantai.html', 'place_num': '59'},
  '潍坊市': {'link': '/lvyou-weifang.html', 'place_num': '71'},
  '济宁市': {'link': '/lvyou-jining.html', 'place_num': '50'},
  '泰安市': {'link': '/lvyou-taian.html', 'place_num': '31'},
  '威海市': {'link': '/lvyou-weihai.html', 'place_num': '44'},
  '日照市': {'link': '/lvyou-rizhao.html', 'place_num': '20'},
  '临沂市': {'link': '/lvyou-linyi.html', 'place_num': '75'},
  '德州市': {'link': '/lvyou-dezhou.html', 'place_num': '21'},
  '聊城市': {'link': '/lvyou-liaocheng.html', 'place_num': '22'},
  '滨州市': {'link': '/lvyou-binzhou.html', 'place_num': '23'},
  '菏泽市': {'link': '/lvyou-heze.h

In [9]:
base_url = 'https://www.meet99.com'

In [10]:
def get_location_url(url):
    strhtml = getHtml(url)
    location_soup = BeautifulSoup(strhtml.text,'lxml')
    location_url_data = location_soup.select('#treemenu > li.open > ul > li.open > ul > li.open > ul > li > a')
    location_info = {}
    for item in location_url_data:
        text = item.get_text()
        link = item.get('href')

        location_info[text[:text.find('(')-1]] = {}
        location_info[text[:text.find('(')-1]]['link'] = link
        location_info[text[:text.find('(')-1]]['place_num'] = text[text.find('(')+1:-1]
    return location_info

    

In [20]:
error_list = []
base_url = 'https://www.meet99.com'
for province_key in city_url_info.keys():
    if province_key not in ['北京市', '天津市', '上海市', '重庆市', '香港', '澳门']:
        for city_key in city_url_info[province_key]['children'].keys():
            url = base_url + city_url_info[province_key]['children'][city_key]['link']
            print(city_key)
            print(url)
            try:
                location_info = get_location_url(url)
                city_url_info[province_key]['children'][city_key]['children'] = location_info
            except Exception as e:
                print(city_key)
                print('获取区域信息错误，错误原因：', e)
                error_info = {}
                error_info['province_key'] = province_key
                error_info['city_key'] = city_key
                error_info['city_info'] = city_url_info[province_key]['children'][city_key]
                error_list.append(error_info)
            time.sleep(5)

            # break
    # break


石家庄市
https://www.meet99.com/lvyou-shijiazhuang.html


唐山市
https://www.meet99.com/lvyou-tangshan.html
秦皇岛市
https://www.meet99.com/lvyou-qinhuangdao.html
邯郸市
https://www.meet99.com/lvyou-handan.html
邢台市
https://www.meet99.com/lvyou-xingtai.html
保定市
https://www.meet99.com/lvyou-baoding.html
张家口市
https://www.meet99.com/lvyou-zhangjiakou.html
承德市
https://www.meet99.com/lvyou-chengde.html
沧州市
https://www.meet99.com/lvyou-cangzhou.html
廊坊市
https://www.meet99.com/lvyou-langfang.html
衡水市
https://www.meet99.com/lvyou-hengshui.html
太原市
https://www.meet99.com/lvyou-taiyuan.html
大同市
https://www.meet99.com/lvyou-datongshi.html
阳泉市
https://www.meet99.com/lvyou-yangquan.html
长治市
https://www.meet99.com/lvyou-changzhi.html
晋城市
https://www.meet99.com/lvyou-jincheng.html
朔州市
https://www.meet99.com/lvyou-shuozhou.html
晋中市
https://www.meet99.com/lvyou-jinzhong.html
运城市
https://www.meet99.com/lvyou-yuncheng.html
忻州市
https://www.meet99.com/lvyou-xinzhou.html
临汾市
https://www.meet99.com/lvyou-linfen.html
吕梁市
https://www.meet99.com/lvyou-lvliang.html
呼和浩特市
https://

In [21]:
error_list

[]

In [38]:
'''
    获取区域的所有景点
'''
def get_place_url(url):
    response = getHtml(url)
    location_place_url_soup=BeautifulSoup(response.text,'lxml')
    location_place_url_data = location_place_url_soup.select('#tiles > li')

    place_info_list = []

    for item in location_place_url_data:

        try:
            place_info={}
            place_info['href'] = item.a.get('href')

            item2 = item.select('li > div.bar')[0]
            place_info['name'] = item2.attrs['m']
            place_info['ever'] = item2.select('a.ever > span')[0].get_text()
            place_info['want'] = item2.select('a.never > span')[0].get_text()

            place_info_list.append(place_info)

        except Exception as e:
            print(e)
    time.sleep(2)
    return place_info_list

In [39]:
'''
    获取景点信息
'''
def get_place_info(url):

    place_response = getHtml(url)
    place_soup=BeautifulSoup(place_response.text,'html5lib')
    place_header_data = place_soup.select('#jdleft > div.info > div > div.hd > ul > li')
    place_data = place_soup.select('#jdleft > div.info > div > div > div.item')

    place_info_item = {}
    for i in range(len(place_header_data)):

        if place_header_data[i].get_text() in ['学校介绍', '景区介绍', '旅游攻略', '怎么去', '门票']:
            # print(place_header_data[i].get_text())
            # print(place_data[i])
            item_header = place_data[i].find_all('h2')
            # print(item_header)
            place_data_text = place_data[i].get_text()
            
            if place_header_data[i].get_text()=='旅游攻略':
                lv_gl_text_data = place_data[i].select('ul')
                if len(lv_gl_text_data) > 0:
                    lv_gl_text = lv_gl_text_data[0].get_text()
                    place_data_text = place_data_text.replace(lv_gl_text, '')
            
            place_data_text = place_data_text.replace('信息纠错', '')
            place_data_text = place_data_text.replace('景区的卫星地图', '')

            

            place_info_item[place_header_data[i].get_text()] = {}

            for j in range(len(item_header)):
                current_header = item_header[j].get_text()[:-1]
                current_text = place_data_text[place_data_text.find(current_header)+len(current_header)+1:]

                if j < len(item_header)-1:
                    next_header = item_header[j+1].get_text()
                    current_text = current_text[:current_text.find(next_header)]
                
                if current_header not in ['景区游览图', '学校地图', '景区地图', '驴友游记']:
                    place_info_item[place_header_data[i].get_text()][current_header] = current_text.replace('　　', '').replace(' ', '').split('\n')
    time.sleep(2)
    
    return place_info_item



In [44]:
!pip3 install html5lib

In [48]:
location_error_list = []
error_list = []
base_url = 'https://www.meet99.com'
rlt = {'name':[], 'province': [], 'city': [], 'location': [], 'desc': [], 'want': [], 'has_been': [], 'strategy': [], 'route': [], 'ticket_info': [], 'open_time': []}

for province_key in city_url_info.keys():
    if province_key not in ['北京市', '天津市', '上海市', '重庆市', '香港', '澳门'] and province_key in ['山东省', '山西省', '河南省', '河北省']:
        for city_key in city_url_info[province_key]['children'].keys():
            # print(city_key)
            
            if 'children' in city_url_info[province_key]['children'][city_key] and len(city_url_info[province_key]['children'][city_key]['children'].keys()) > 0 :
                
                for location_key in city_url_info[province_key]['children'][city_key]['children'].keys():
                    location_info = city_url_info[province_key]['children'][city_key]['children'][location_key]
                    location_url = base_url + location_info['link']
                    print(location_key)
                    print(location_url)
                    location_place_info_list = []
                    try:
                        location_place_info_list = get_place_url(location_url)

                    except Exception as e:
                        print('获取区域内的景点出现异常,异常原因:', e)
                        print(location_info)
                        location_error_info = {}
                        location_error_info['province_key'] = province_key
                        location_error_info['city_key'] = city_key
                        location_error_info['location_name'] = location_key
                        location_error_info['location_info'] = location_info
                        location_error_list.append(location_error_info)


                    for place_item in location_place_info_list:
                        try:
                            print(place_item)
                            place_url = base_url + place_item['href']
                            place_info = get_place_info(place_url)

                            name = place_item['name']
                            
                            province = province_key
                            city = city_key
                            location = location_key
                            desc = {}
                            if '景区介绍' in place_info.keys():
                                desc = place_info['景区介绍']
                            elif '学校介绍' in place_info.keys():
                                desc = place_info['学校介绍']

                            want = place_item['want']
                            has_been = place_item['ever']

                            strategy = {}
                            if '旅游攻略' in place_info.keys():
                                strategy = place_info['旅游攻略']

                            route = {}
                            if '怎么去' in place_info.keys():
                                route = place_info['怎么去']

                            ticket_info = {}
                            open_time = {}
                            if '门票' in place_info.keys():
                                for key in place_info['门票']:
                                    if key not in ['景区开放时间']:
                                        ticket_info[key] = place_info['门票'][key]


                                if '景区开放时间' in place_info['门票'].keys():
                                    open_time = place_info['门票']['景区开放时间']
                            
                            ticket = ticket_info

                            rlt['name'].append(name)
                            rlt['province'].append(province)
                            rlt['city'].append(city)
                            rlt['location'].append(location)
                            rlt['desc'].append(desc)
                            rlt['want'].append(want)
                            rlt['has_been'].append(has_been)
                            rlt['strategy'].append(strategy)
                            rlt['route'].append(route)
                            rlt['ticket_info'].append(ticket_info)
                            rlt['open_time'].append(open_time)
                            

                        except Exception as e:
                            print('以下景点出现异常,异常原因:', e)
                            print(place_item)
                            error_info = {}
                            error_info['province_key'] = province_key
                            error_info['city_key'] = city_key
                            error_info['location_name'] = location_key
                            error_info['place_info'] = place_item
                            error_list.append(error_info)  

                time.sleep(5)

                    


正定县
https://www.meet99.com/lvyou-zhengding.html


'NoneType' object has no attribute 'get'
{'href': '/jingdian-longxingsi.html', 'name': '正定隆兴寺', 'ever': '2275', 'want': '475'}
以下景点出现异常,异常原因: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?
{'href': '/jingdian-longxingsi.html', 'name': '正定隆兴寺', 'ever': '2275', 'want': '475'}
{'href': '/jingdian-rongguofu.html', 'name': '正定荣国府', 'ever': '775', 'want': '119'}
以下景点出现异常,异常原因: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?
{'href': '/jingdian-rongguofu.html', 'name': '正定荣国府', 'ever': '775', 'want': '119'}
{'href': '/jingdian-zhaoyunmiao.html', 'name': '正定赵云庙', 'ever': '310', 'want': '49'}
以下景点出现异常,异常原因: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?
{'href': '/jingdian-zhaoyunmiao.html', 'name': '正定赵云庙', 'ever': '310', 'want': '49'}
{'href': '/jingdian-changlemen.html', 'name': '正定长乐门', 'ever': '1

KeyboardInterrupt: 

In [8]:
rlt

NameError: name 'rlt' is not defined